<a href="https://colab.research.google.com/github/olfabre/DataMaining_Master_2/blob/main/TP2_1_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TP2_1 - Panorama des méthodes de fouille de données - Analyse exploratoire

Charger le fichier pokemon.csv

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving pokemon.csv to pokemon.csv


On ouvre le fichier pokemon.csv à l’aide de la bibliothèque pandas.

In [ ]:
import pandas as pd

# 1. Chargement du jeu de données
df = pd.read_csv('pokemon.csv')

# 2. Nombre d’attributs (colonnes)
nb_attributs = df.shape[1]
print("Nombre d’attributs :", nb_attributs)

# 3. Types de chaque attribut
types_attributs = df.dtypes
print("\nTypes des attributs :")
print(types_attributs)

# 4. Nombre total de Pokémons recensés
nb_pokemons_total = df.shape[0]
print("\nNombre total de Pokémons :", nb_pokemons_total)

# 5. Conservation uniquement de la 1re génération
df_gen1 = df[df['generation'] == 1].copy()
nb_gen1 = df_gen1.shape[0]
print("\nNombre de Pokémons de 1re génération :", nb_gen1)

# 6. Suppression des colonnes non désirées
colonnes_a_garder = ['name', 'type1'] + \
                    [col for col in df_gen1.columns
                     if df_gen1[col].dtype in ['int64', 'float64'] and col != 'generation']

df_final = df_gen1[colonnes_a_garder]

# Aperçu du résultat
print("\nAperçu du DataFrame final :")
df_final.head()

Nombre d’attributs : 41

Types des attributs :
abilities             object
against_bug          float64
against_dark         float64
against_dragon       float64
against_electric     float64
against_fairy        float64
against_fight        float64
against_fire         float64
against_flying       float64
against_ghost        float64
against_grass        float64
against_ground       float64
against_ice          float64
against_normal       float64
against_poison       float64
against_psychic      float64
against_rock         float64
against_steel        float64
against_water        float64
attack                 int64
base_egg_steps         int64
base_happiness         int64
base_total             int64
capture_rate          object
classfication         object
defense                int64
experience_growth      int64
height_m             float64
hp                     int64
japanese_name         object
name                  object
percentage_male      float64
pokedex_number         in

,name,type1,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,...,experience_growth,height_m,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,weight_kg,is_legendary
0,Bulbasaur,grass,1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,...,1059860,0.7,45,88.1,1,65,65,45,6.9,0
1,Ivysaur,grass,1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,...,1059860,1.0,60,88.1,2,80,80,60,13.0,0
2,Venusaur,grass,1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,...,1059860,2.0,80,88.1,3,122,120,80,100.0,0
3,Charmander,fire,0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,...,1059860,0.6,39,88.1,4,60,50,65,8.5,0
4,Charmeleon,fire,0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,...,1059860,1.1,58,88.1,5,80,65,80,19.0,0


Création d'un fichier csv ne comportant que les pokemons de première génration

In [ ]:
"""
pokemons_first.py
Crée pokemons_first.csv :
- uniquement les Pokémons de 1ʳᵉ génération
- sans la colonne « generation »
- sans les colonnes non-numériques, sauf « name » et « type1 »
Affiche le nombre d'instances restantes.
"""

import pandas as pd

# --------- paramètres ----------
FICHIER_ENTREE  = "pokemon.csv"      # ou "pokemons.csv"
FICHIER_SORTIE  = "pokemons_first.csv"
# -------------------------------

def main():
    # 1. Chargement
    df = pd.read_csv(FICHIER_ENTREE)

    # 2. Filtre 1ʳᵉ génération
    df = df[df["generation"] == 1].copy()

    # 3. Colonnes à conserver
    cols_a_garder = ["name", "type1"] + \
                    [c for c in df.columns
                     if pd.api.types.is_numeric_dtype(df[c]) and c != "generation"]

    df_final = df[cols_a_garder]

    # 4. Export
    df_final.to_csv(FICHIER_SORTIE, index=False)

    # 5. Affichage du nombre d'instances
    print(f"Nombre de Pokémons de 1ʳᵉ génération : {len(df_final)}")
    print(f"Fichier créé : {FICHIER_SORTIE}")

if __name__ == "__main__":
    main()

Nombre de Pokémons de 1ʳᵉ génération : 151
Fichier créé : pokemons_first.csv


Y a-t-il beaucoup de valeurs manquantes ?

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.read_csv('pokemons_first.csv')

manquantes = df.isna().sum().sort_values(ascending=False)
print(manquantes[manquantes > 0])

weight_kg          18
height_m           18
percentage_male    13
dtype: int64


Remplacer les valeurs manquantes par la valeur calculée par la stratégie = mdédian



In [ ]:
# impute_median.py
import pandas as pd
from sklearn.impute import SimpleImputer

# 1. chargement
file = "pokemons_first.csv"
df = pd.read_csv(file)

print("Valeurs manquantes AVANT :", df.isna().sum().sum())

# 2. colonnes numériques
cols_num = df.select_dtypes(include=['float64', 'int64']).columns

# 3. imputation par la médiane
imputer = SimpleImputer(strategy='median')
df[cols_num] = imputer.fit_transform(df[cols_num])

print("Valeurs manquantes APRÈS :", df.isna().sum().sum())

# 4. sauvegarde (même fichier)
df.to_csv(file, index=False)
print(f"Fichier '{file}' mis à jour avec les médianes.")

Valeurs manquantes AVANT : 49
Valeurs manquantes APRÈS : 0
Fichier 'pokemons_first.csv' mis à jour avec les médianes.


On va essayer de regarder rapidement les écarts entre les données

In [ ]:
df.describe().loc[['mean','std']]

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,experience_growth,height_m,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,weight_kg,is_legendary
mean,0.905629,1.066225,0.980132,1.158940,0.917219,0.953642,1.036424,1.168874,0.966887,1.147351,...,1.052281e+06,1.224503,64.344371,55.466887,76.000000,69.403974,67.741722,70.152318,43.941060,0.033113
std,0.572088,0.349645,0.237212,0.693712,0.380484,0.560062,0.620750,0.573403,0.508818,0.970941,...,1.018366e+05,0.944032,28.593483,20.557275,43.734045,33.455080,25.974722,29.170018,54.713617,0.179526
